# Modèle de survie optimisé pour prédiction du risque de décès
Ce notebook combine un feature engineering avancé, plusieurs modèles de survie spécialisés et un ensemble automatique pour maximiser le score IPCW-C-index tout en évitant l'overfitting.

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sksurv.ensemble import RandomSurvivalForest
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.metrics import concordance_index_censored
from lifelines import CoxPHFitter
import warnings
warnings.filterwarnings('ignore')

## 1. Chargement des données

In [7]:
clin = pd.read_csv('X_train/clinical_train.csv')
mol = pd.read_csv('X_train/molecular_train.csv')
target = pd.read_csv('target_train.csv')
clin_test = pd.read_csv('X_test/clinical_test.csv')
mol_test = pd.read_csv('X_test/molecular_test.csv')

## 2. Feature engineering avancé (clinique et moléculaire)

In [8]:
def advanced_clinical_features(df):
    df = df.copy()
    num_cols = ['BM_BLAST', 'WBC', 'ANC', 'MONOCYTES', 'HB', 'PLT']
    for col in num_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df[num_cols] = SimpleImputer(strategy='median').fit_transform(df[num_cols])
    df['log_WBC'] = np.log1p(df['WBC'])
    df['log_ANC'] = np.log1p(df['ANC'])
    df['log_MONOCYTES'] = np.log1p(df['MONOCYTES'])
    df['log_PLT'] = np.log1p(df['PLT'])
    df['hemato_risk_score'] = ((df['BM_BLAST'] > 20).astype(int) * 2 + (df['WBC'] > 100).astype(int) + (df['PLT'] < 50).astype(int) + (df['HB'] < 8).astype(int))
    df['organ_dysfunction'] = ((df['HB'] < 8).astype(int) + (df['PLT'] < 20).astype(int) + (df['ANC'] < 0.5).astype(int))
    df['blast_to_wbc_ratio'] = df['BM_BLAST'] / (df['WBC'] + 1)
    df['anc_to_wbc_ratio'] = df['ANC'] / (df['WBC'] + 1)
    df['monocyte_to_wbc_ratio'] = df['MONOCYTES'] / (df['WBC'] + 1)
    def extract_cytogenetic_features(cyto_str):
        if pd.isna(cyto_str): return {'cyto_normal': 0, 'cyto_complex': 0, 'cyto_monosomy7': 0, 'cyto_trisomy8': 0, 'cyto_inv16': 0, 'cyto_t821': 0, 'cyto_del5q': 0, 'cyto_del20q': 0, 'cyto_abnormalities_count': 0} 
        cyto_str = str(cyto_str).lower()
        abnormality_markers = ['+', '-', 'del', 'inv', 't(', 'ins', 'dup']
        abnormality_count = sum(cyto_str.count(marker) for marker in abnormality_markers)
        return {
            'cyto_normal': 1 if ('46,xx' in cyto_str or '46,xy' in cyto_str) and len(cyto_str) < 20 else 0,
            'cyto_complex': 1 if abnormality_count >= 3 else 0,
            'cyto_abnormalities_count': abnormality_count,
            'cyto_monosomy7': 1 if '-7' in cyto_str or 'del(7' in cyto_str else 0,
            'cyto_trisomy8': 1 if '+8' in cyto_str else 0,
            'cyto_inv16': 1 if 'inv(16)' in cyto_str else 0,
            'cyto_t821': 1 if 't(8;21)' in cyto_str else 0,
            'cyto_del5q': 1 if 'del(5q)' in cyto_str or '-5' in cyto_str else 0,
            'cyto_del20q': 1 if 'del(20q)' in cyto_str else 0
        }
    cyto_features = df['CYTOGENETICS'].apply(extract_cytogenetic_features)
    cyto_df = pd.DataFrame(cyto_features.tolist())
    df = pd.concat([df, cyto_df], axis=1)
    def calculate_cytogenetic_risk(row):
        if row['cyto_inv16'] or row['cyto_t821']: return 0
        elif row['cyto_complex'] or row['cyto_monosomy7'] or row['cyto_del5q']: return 2
        else: return 1
    df['cytogenetic_risk_group'] = df.apply(calculate_cytogenetic_risk, axis=1)
    df['blast_hemoglobin_interaction'] = df['BM_BLAST'] * (1 / (df['HB'] + 1))
    df['wbc_plt_interaction'] = df['log_WBC'] * (1 / (df['log_PLT'] + 1))
    df['high_blast'] = (df['BM_BLAST'] > 30).astype(int)
    df['very_high_wbc'] = (df['WBC'] > 100).astype(int)
    df['severe_anemia'] = (df['HB'] < 8).astype(int)
    df['severe_thrombocytopenia'] = (df['PLT'] < 20).astype(int)
    df['severe_neutropenia'] = (df['ANC'] < 0.5).astype(int)
    return df

In [9]:
def advanced_molecular_features(mol_df):
    mol_df = mol_df.copy()
    mol_df['VAF'] = pd.to_numeric(mol_df['VAF'], errors='coerce')
    mutation_counts = mol_df.groupby('ID').size().rename('total_mutations')
    vaf_stats = mol_df.groupby('ID')['VAF'].agg([('vaf_mean', 'mean'), ('vaf_median', 'median'), ('vaf_max', 'max'), ('vaf_min', 'min'), ('vaf_std', 'std')]).fillna(0)
    high_impact_effects = ['stop_gained', 'frameshift_variant', 'splice_site_variant', 'start_lost', 'stop_lost', 'transcript_ablation']
    moderate_impact_effects = ['missense_variant', 'inframe_deletion', 'inframe_insertion', 'protein_altering_variant']
    mol_df['high_impact'] = mol_df['EFFECT'].isin(high_impact_effects).astype(int)
    mol_df['moderate_impact'] = mol_df['EFFECT'].isin(moderate_impact_effects).astype(int)
    impact_counts = mol_df.groupby('ID').agg({'high_impact': 'sum', 'moderate_impact': 'sum'}).rename(columns={'high_impact': 'high_impact_mutations', 'moderate_impact': 'moderate_impact_mutations'})
    top_genes = mol_df['GENE'].value_counts().head(10).index.tolist()
    for gene in top_genes:
        mol_df[gene] = (mol_df['GENE'] == gene).astype(int)
    gene_features = mol_df.groupby('ID')[top_genes].max() if top_genes else pd.DataFrame()
    molecular_features = pd.concat([mutation_counts, vaf_stats, impact_counts, gene_features], axis=1).fillna(0)
    return molecular_features

In [10]:
# Application du feature engineering
clin_train = advanced_clinical_features(clin)
clin_test_proc = advanced_clinical_features(clin_test)
mol_train = advanced_molecular_features(mol)
mol_test_proc = advanced_molecular_features(mol_test)

In [11]:
# Encodage des centres
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
ctr_train = ohe.fit_transform(clin_train[['CENTER']].fillna('Unknown'))
ctr_test = ohe.transform(clin_test_proc[['CENTER']].fillna('Unknown'))
ctr_train_df = pd.DataFrame(ctr_train, columns=ohe.get_feature_names_out(['CENTER']))
ctr_test_df = pd.DataFrame(ctr_test, columns=ohe.get_feature_names_out(['CENTER']))
clin_train = pd.concat([clin_train.reset_index(drop=True), ctr_train_df], axis=1)
clin_test_proc = pd.concat([clin_test_proc.reset_index(drop=True), ctr_test_df], axis=1)
clin_train.drop(['CENTER','CYTOGENETICS'], axis=1, inplace=True)
clin_test_proc.drop(['CENTER','CYTOGENETICS'], axis=1, inplace=True)

In [12]:
# Fusion des données
X_train = clin_train.set_index('ID').join(mol_train, how='left').fillna(0)
X_test = clin_test_proc.set_index('ID').join(mol_test_proc, how='left').fillna(0)
missing_cols = set(X_train.columns) - set(X_test.columns)
for col in missing_cols: X_test[col] = 0
extra_cols = set(X_test.columns) - set(X_train.columns)
X_test = X_test.drop(columns=list(extra_cols))
X_test = X_test[X_train.columns]

In [13]:
# Normalisation
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

In [14]:
# Préparation de la cible
target = target.dropna(subset=['OS_YEARS','OS_STATUS'])
y_survival = np.array([(bool(s), t) for s, t in zip(target['OS_STATUS'], target['OS_YEARS'])], dtype=[('event', bool), ('time', float)])
ids = target['ID'].values
X_train_scaled = X_train_scaled.loc[ids]

## 3. Entraînement des modèles de survie et ensemble

In [15]:
# Cox ElasticNet
cox_model = CoxnetSurvivalAnalysis(l1_ratio=0.5, alpha_min_ratio=0.01, max_iter=1000)
cox_model.fit(X_train_scaled, y_survival)
cox_risk_scores = cox_model.predict(X_train_scaled)
cox_test_scores = cox_model.predict(X_test_scaled)
cox_c_index = concordance_index_censored(y_survival['event'], y_survival['time'], cox_risk_scores)[0]
# Random Survival Forest
rsf_model = RandomSurvivalForest(n_estimators=500, max_depth=10, min_samples_split=10, min_samples_leaf=5, random_state=42, n_jobs=-1)
rsf_model.fit(X_train_scaled, y_survival)
rsf_risk_scores = rsf_model.predict(X_train_scaled)
rsf_test_scores = rsf_model.predict(X_test_scaled)
rsf_c_index = concordance_index_censored(y_survival['event'], y_survival['time'], rsf_risk_scores)[0]

In [16]:
# Ensemble des deux meilleurs modèles
risk_scaler = MinMaxScaler()
train_scores = np.column_stack([cox_risk_scores, rsf_risk_scores])
test_scores = np.column_stack([cox_test_scores, rsf_test_scores])
train_scores_norm = risk_scaler.fit_transform(train_scores)
test_scores_norm = risk_scaler.transform(test_scores)
ensemble_strategies = {
    'mean': np.mean(test_scores_norm, axis=1),
    'weighted_mean': np.average(test_scores_norm, weights=[cox_c_index, rsf_c_index], axis=1),
    'median': np.median(test_scores_norm, axis=1),
    'rsf_only': test_scores_norm[:, 1],
    'cox_only': test_scores_norm[:, 0]
}
# Sélection de la meilleure stratégie sur le train
best_strategy_perf = {}
for strat, _ in ensemble_strategies.items():
    if strat == 'mean': pred = np.mean(train_scores_norm, axis=1)
    elif strat == 'weighted_mean': pred = np.average(train_scores_norm, weights=[cox_c_index, rsf_c_index], axis=1)
    elif strat == 'median': pred = np.median(train_scores_norm, axis=1)
    elif strat == 'rsf_only': pred = train_scores_norm[:, 1]
    else: pred = train_scores_norm[:, 0]
    best_strategy_perf[strat] = concordance_index_censored(y_survival['event'], y_survival['time'], pred)[0]
best_strategy = max(best_strategy_perf, key=best_strategy_perf.get)
final_test_scores = ensemble_strategies[best_strategy]

## 4. Génération du fichier de soumission

In [ ]:
submission = pd.DataFrame({'ID': X_test_scaled.index, 'risk_score': final_test_scores})
submission = submission.drop_duplicates(subset=['ID'])
submission.to_csv('y_test.csv', index=False)
print(submission.head())

# SCORE : 0.7479

     ID  risk_score
0  KYW1    0.637653
1  KYW2    0.642835
2  KYW3    0.356406
3  KYW4    0.525644
4  KYW5    0.570518
